In [5]:
import pandas as pd
from datetime import datetime
import re
import numpy as np
import plotly.express as px
import sys
import os
sys.path.append(os.path.abspath('../'))
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.svm import SVC, SVR
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score
from sklearn.metrics import mean_squared_error as MSE, mean_absolute_error as MAE

from helper_funcs import get_data, convert_unix_to_datetime, separate_symbols
import sqlite3
from helper_funcs import model_pipeline, preprocess

In [6]:
DB_PATH = os.environ.get('DB_PATH')
DB_PATH = os.path.realpath(f'../../{DB_PATH}')

In [7]:
tickers = ['BTC', 'ETH']
data = get_data(tickers, DB_PATH)

df = pd.DataFrame(data, columns = ['id', 'symbol', 'date', 'high', 'low', 'open', 'close', 'volumeto', 'volumefor'])
df = df.drop('id', axis=1)
df['date'] = convert_unix_to_datetime(df['date'])
btc, eth  = separate_symbols(df)

/Users/iandias/Desktop/Quant/algo-trading/historical_data.db


In [8]:
btc_process = preprocess(btc, 'BTC')
btc_process = btc_process.dropna()

/Users/iandias/Desktop/Quant/algo-trading/notebooks/helper_funcs.py:119: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['close_lag'+str(i)] = lag_close_col
/Users/iandias/Desktop/Quant/algo-trading/notebooks/helper_funcs.py:120: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['parkinson_lag'+str(i)] = lag_park_col
/Users/iandias/Desktop/Quant/algo-trading/notebooks/helper_funcs.py:119: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

In [9]:
model = model_pipeline(btc_process)
btc_model = model.make_dataframe()
btc_model

/Users/iandias/Desktop/Quant/algo-trading/.direnv/python-3.8.10/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/iandias/Desktop/Quant/algo-trading/.direnv/python-3.8.10/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/iandias/Desktop/Quant/algo-trading/.direnv/python-3.8.10/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

,MAE,MSE,f1,precision,recall,accuracy
LogisticRegression,NaN,NaN,0.0,0.0,0.0,0.810498
RandomForestClassifier,NaN,NaN,0.148148,0.6,0.084507,0.815836
SVC,NaN,NaN,0.0,0.0,0.0,0.810498
EnsembleClf,NaN,NaN,0.0,0.0,0.0,0.810498
LinearRegression,0.008572,0.000563,NaN,NaN,NaN,NaN
RandomForestRegressor,0.018251,0.001014,NaN,NaN,NaN,NaN
SVR,0.534005,0.4054,NaN,NaN,NaN,NaN
EnsembleReg,0.180185,0.04632,NaN,NaN,NaN,NaN


In [10]:
_, pred, _ = model.fit_model('RandomForestClassifier')

In [13]:
sum(pred) / len(pred)

0.030249110320284697

In [14]:
btc.head()

,symbol,date,high,low,open,close,volumeto,volumefor,rel_price_change,parkinson_vol
0,BTC,2016-11-08,714.63,700.96,705.4,6.566532,28262235.71,39732.33,0.019314,0.008040
4002,BTC,2016-11-08,714.63,700.96,705.4,6.566532,28262235.71,39732.33,0.019314,0.008040
2001,BTC,2016-11-08,714.63,700.96,705.4,6.581221,28262235.71,39732.33,0.019314,0.008040
1,BTC,2016-11-09,741.23,707.13,710.9,6.581221,51548567.54,70698.20,0.047088,0.019605
4003,BTC,2016-11-09,741.23,707.13,710.9,6.581221,51548567.54,70698.20,0.047088,0.019605


In [ ]:
import backtrader as bt
import backtrader.analyzers as btanalyzers

In [ ]:
class backtesting(bt.Strategy):
    def __init__(self):
        ma_fast = bt.ind.SMA(period = 10)
        ma_slow = bt.ind.SMA(period = 50)
        
        self.crossover = bt.ind.CrossOver(ma_fast, ma_slow)
        
    
    def next(self):
        if not self.position:
            if self.crossover > 0:
                self.buy()
        elif self.crossover < 0:
            self.close()
        

In [ ]:

btc.drop('symbol', axis=1, inplace=True)
btc = btc.dropna()
btc.set_index('date', inplace=True)

In [ ]:
btc_bt = btc[['open', 'high', 'low', 'close', 'volumeto']]
btc_bt.rename(columns={'volumeto': 'volume'}, inplace=True)

In [ ]:
pred = [int(x) for x in pred
pred = pd.DataFrame(pred, index = btc_bt[-374:].index, columns=['predicted'])

In [ ]:
prices = pred.join(btc_bt[-374:])

In [ ]:
prices.index = pd.to_datetime(prices.index)

In [ ]:
OHLCV = ['open', 'high', 'low', 'close', 'volume']
class SignalData(bt.feeds.PandasData):
    """
    Define pandas DataFrame structure
    """
    cols = OHLCV + ['predicted']
    # create lines
    lines = tuple(cols)
    
    # define parameters
    params = {c: -1 for c in cols}
    params.update({'datetime': None})
    params = tuple(params.items())

In [ ]:
# define backtesting strategy class
class MLStrategy(bt.Strategy):
    params = dict(
    )
    
    def __init__(self):
        # keep track of open, close prices and predicted value in the series
        self.data_predicted = self.datas[0].predicted
        self.data_open = self.datas[0].open
        self.data_close = self.datas[0].close
        
        # keep track of pending orders/buy price/buy commission
        self.order = None
        self.price = None
        self.comm = None    # logging function
    def log(self, txt):
        '''Logging function'''
        dt = self.datas[0].datetime.date(0).isoformat()
        print(f'{dt}, {txt}')    
    
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # order already submitted/accepted - no action required
            return        # report executed order
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(f'BUY EXECUTED --- Price: {order.executed.price:.2f}, Cost: {order.executed.value:.2f},Commission: {order.executed.comm:.2f}'
                )
                self.price = order.executed.price
                self.comm = order.executed.comm
            else:
                self.log(f'SELL EXECUTED --- Price: {order.executed.price:.2f}, Cost: {order.executed.value:.2f},Commission: {order.executed.comm:.2f}'
                )        # report failed order
        elif order.status in [order.Canceled, order.Margin, 
                              order.Rejected]:
            self.log('Order Failed')        # set no pending order
        self.order = None    
        
    def notify_trade(self, trade):
        if not trade.isclosed:
            return
        
        self.log(f'OPERATION RESULT --- Gross: {trade.pnl:.2f}, Net: {trade.pnlcomm:.2f}')    
        # We have set cheat_on_open = True.This means that we calculated the signals on day t's close price, 
        # but calculated the number of shares we wanted to buy based on day t+1's open price.
    
    def next_open(self):
        if not self.position:
            if self.data_predicted > 0:
                # calculate the max number of shares ('all-in')
                size = int(self.broker.getcash() / self.datas[0].open) / 10
                # buy order
                self.log(f'BUY CREATED --- Size: {size}, Cash: {self.broker.getcash():.2f}, Open: {self.data_open[0]}, Close: {self.data_close[0]}')
                self.buy(size=size)
        else:
            if self.data_predicted < 0:
                # sell order
                self.log(f'SELL CREATED --- Size: {self.position.size}')
                self.sell(size=self.position.size)

In [ ]:
# instantiate SignalData class
data = SignalData(dataname=prices)# instantiate Cerebro, add strategy, data, initial cash, commission and pyfolio for performance analysis
cerebro = bt.Cerebro(stdstats = False, cheat_on_open=True)
cerebro.addstrategy(MLStrategy)
cerebro.adddata(data)
cerebro.broker.setcash(100000.0)
cerebro.broker.setcommission(commission=0.001)
#cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyfolio')# run the backtest
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
backtest_result = cerebro.run()
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())


In [ ]:
cerebro = bt.Cerebro()
#data = bt.feeds.YahooFinanceData(dataname='AAPL', fromdate = datetime(2010, 1, 1), todate = datetime(2020, 1, 1))
data = bt.feeds.PandasData(dataname=btc)
cerebro.adddata(data)

In [ ]:
cerebro.addstrategy(backtesting)
cerebro.broker.setcash(1000.0)
cerebro.addsizer(bt.sizers.PercentSizer, percents=10)

In [ ]:
cerebro.addanalyzer(btanalyzers.SharpeRatio, _name = "sharpe")
cerebro.addanalyzer(btanalyzers.Transactions, _name = "transactions")
cerebro.addanalyzer(btanalyzers.TradeAnalyzer, _name = "trades")

In [ ]:
back = cerebro.run()

In [ ]:
cerebro.broker.getvalue()

In [ ]:
back[0].analyzers.sharpe.get_analysis()
back[0].analyzers.transactions.get_analysis()
back[0].analyzers.trades.get_analysis()